## 12 - Classifying Detections using ipywidgets

In order to streamline classifications we have developed a GUI using the ipywidgets package to quickly scan through ELixer report summaries and store source classifications. If you do not have previous experience classifying with these reports, we highly recommend you read the Elixer_readme.pdf located in 
```
/work/03946/hetdex/hdr1/software/elixer/docs/Elixer_readme.pdf
```
ELiXer reports allow us to visually classify HETDEX line emitting galaxies by combining HETDEX fiber spectra and 2D CCD images of line detections with ancillary photometric images and catalogs. You only need to know the detectid of a source to look up the ELiXer report.

The GUI will take either a detectlist in the form of a numpy array, a saved detectlist (saved previously using np.savetxt), or a saved classification table file. You can only classify objects within this input list. You may also specify the name of the output file for the classification table.

You can overwrite classifications that you aren't happy with and inspect your information along the way if you like. We will show you how in this notebook.

### Launch the widget

The widget is launched by calling the class `ElixerWidget()` from `hetdex_api.elixer_widgets`. When you launch the widget you are generating a dictionary to store your visual classifications as well as initializing the widget. It either loads in a pre-defined detection list or if no argument is given it loads a list of all HDR1 detections. For our first team classification effort, please use the DetectID list stored in `detects_hdr1_sample1.txt`[TBD]. We'll also show a simple query here to get you started, but to learn more about the Detections API please go to Notebook 03 so you can do your own queries or you can explore the query interface to the Detections database in Notebook 11. 

Some quick notes about the interface:

    * the DetectID text box will take any integer value so you can open any HDR1 detectID on demand
    
    * The buttons "Previous DetectID" and "Next DetectID" will jump through the input detectlist
    
    * if that DetectID is not in the input detectlist, then it will open the elixer but you will not be 
      able to save the classification
    
    * to return to your input list just press the "Next DetectID" button or "Previous DetectID" button and 
      it will go to the closest DetectID to the value in the textbox
    
    * it is better to leave a classification blank if you are very unsure about it. Classify a detection 
      as Junk that does not seem real and should not be used in any classification algorithms. Just go to 
      the next source by pressing the `Next DetectID` button. Feel free to leave a comment if you like
    
    * the comments box can be used to note bad amps or pixel defects. Either say 'bad amp' or 'bad pixel' 
      or just 'bad' to help us search for it later.

First run this to prevent scrolling in individual cells in the jupyter notebook

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
from hetdex_api.elixer_widgets import *
from hetdex_api.detections import *

## Intialize the Detections class

If you are doing team classifications using the input file `detects_hdr1_sample1.txt`, you do not need to do this step. Initializing the Detections class takes ~10-15 seconds.

In [3]:
detects = Detections('hdr1').refine()

For example here is a selection that would help us find a sample of nice bright LAEs, and of course some interloping OII emitters:

In [4]:
sel = (detects.gmag > 23) * (detects.sn > 10) * (detects.date >= 20171201) * (detects.throughput > 0.095) * (detects.fwhm < 2.6) * (detects.continuum < 10)
np.sum(sel)

2955

The Detections class object can be indexed with a boolean mask. It can work on both just a single array attribute or the entire class (see Notebook 03 for more).

In [5]:
detectLAEcandidates = detects.detectid[sel]
np.savetxt('detectLAEs.txt', detectLAEcandidates)

To open the GUI to just explore all detectIDs and ELiXeR reports:

In [6]:
# elix_widget = ElixerWidget() 

To open the GUI to explore the LAE candidates defined in the previous query or any input detectid list:

In [7]:
elix_widget = ElixerWidget(detectlist=detectLAEcandidates)

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEJvdW5kZWRJbnRUZXh0KHZhbHVlPTEwMDAwODg3MTAsIGRlc2NyaXB0aW9uPXUnRGV0ZWN0SUQ6JywgbWF4PTEwMDA2OTA3OTksIG1pbj0xMDAwMDAwMDDigKY=


or if you saved a list previously in another notebook, you can open it like this:

In [9]:
# elix_widget = ElixerWidget(detectfile='detectLAEs.txt')

Or if you have done some classifications previously and want to continue with the list (by default this list is saved as `elixer_classifications.dat`). If you use the resume=True flag, the detectID will start after the highest detectID classified previously. 

In [10]:
# elix_widget = ElixerWidget(savedfile='elixer_classifications.dat', resume=True)

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEJvdW5kZWRJbnRUZXh0KHZhbHVlPTEwMDAwOTQwNjMsIGRlc2NyaXB0aW9uPXUnRGV0ZWN0SUQ6JywgbWF4PTEwMDA2OTA3OTksIG1pbj0xMDAwMDAwMDDigKY=


## Where the classifications are temporarily stored

`elix_widget` is a class object that contains arrays of the input detectid, vis_class and comment values output from the above GUI. It can be saved and shared with the group using the `Save Progress` button above. It saves

In [11]:
elix_widget.detectid

array([1000088710, 1000088724, 1000088808, ..., 1000689024, 1000689101,
       1000689627])

In [12]:
elix_widget.vis_class

array([ 2,  2,  2, ..., -1, -1, -1])

In [13]:
elix_widget.vis_type

array(['LAE Galaxy', 'LAE Galaxy', 'LAE Galaxy', ..., '0', '0', '0'],
      dtype='|S15')

In [14]:
elix_widget.comment

array(['0', 'test', '0', ..., '0', '0', '0'], dtype='|S30')

When you press the `Save Progress` button, the output is ingested into an astropy Table object and then saved to a txt file. The file name will be the savedfile name if provided to the widget upon intialization. Or you can provide the file name as outfile='filename.dat' as an argument upon initialization. By default, it will save the table to `elixer_classifications.dat`

In [15]:
elix_widget.output

detectid,vis_type,vis_class,comments
int64,str15,int64,str30
1000088710,LAE Galaxy,2,0
1000088724,LAE Galaxy,2,test
1000088808,LAE Galaxy,2,0
1000089022,LAE Galaxy,2,0
1000089039,LAE Galaxy,2,0
1000089053,LAE Galaxy,2,0
1000089322,OII Galaxy,1,0
1000090123,LAE Galaxy,2,0
1000092169,LAE Galaxy,2,0
